In [57]:
####Heterogeneous Infomation Network Embbedding with Variable order Tucker Decomposition####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import itertools
import scipy
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from datetime import time, datetime, timedelta
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
import re
import MeCab
import neologdn
import sys

np.random.seed(98537)

In [58]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no, pattern):
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    return z_id
    if pattern==1:
        Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
        return z_id, Z

In [59]:
####データの設定####
##シナリオの読み込みとクレンジング
#データの読み込み
company = pd.read_csv("D:/Statistics/data/scenario/company.csv")
scenario_df1 = pd.read_csv("D:/Statistics/data/scenario/level1_scenario.csv")
scenario_df2 = pd.read_csv("D:/Statistics/data/scenario/level2_scenario.csv")
factor_df = pd.read_csv("D:/Statistics/data/scenario/Factor.csv")

#読み込んだデータをクレンジング
scenario_df1.columns = (scenario_df1.columns).str.replace(" ", "")
scenario_df2.columns = (scenario_df2.columns).str.replace(" ", "")
factor_df.columns = (factor_df.columns).str.replace(" ", "")
company.columns = (company.columns).str.replace(" ", "")
columns1 = np.array(scenario_df1.columns)
columns2 = np.array(scenario_df2.columns)
columns3 = np.array(factor_df.columns)
columns4 = np.array(company.columns)
for j in range(columns1.shape[0]):
    scenario_df1[columns1[j]] = scenario_df1[columns1[j]].str.replace("\"| ", "")
for j in range(columns2.shape[0]):
    scenario_df2[columns2[j]] = scenario_df2[columns2[j]].str.replace("\"| ", "")
for j in range(columns3.shape[0]):
    factor_df[columns3[j]] = factor_df[columns3[j]].str.replace("\"| ", "")
scenario_df1["c.ticker"] = np.array(scenario_df1["c.ticker"], dtype="int")
scenario_df2["c.ticker"] = np.array(scenario_df2["c.ticker"], dtype="int")
scenario_df1["c.name"] = pd.merge(scenario_df1[["c.ticker"]], company, on="c.ticker", how="left")["c.name"]
scenario_df2["c.name"] = pd.merge(scenario_df2[["c.ticker"]], company, on="c.ticker", how="left")["c.name"]
scenario_df21 = scenario_df2[["f2.area", "f2.predicate", "f2.item", "f2.trend", "f2.element", "a.name", "a.vector", "c.name", "c.ticker"]]
scenario_df21 = scenario_df21.rename(columns={"f2.area": "f.area", "f2.predicate": "f.predicate", 
                                              "f2.item": "f.item", "f2.trend": "f.trend", "f2.element": "f.element"})
scenario_df1 = pd.concat((scenario_df1, scenario_df21), axis=0)
del scenario_df21

In [5]:
##解析対象のレコードを抽出して、名寄を行う
#scenario1のレコードを抽出
index_dup = np.array(np.where(scenario_df1[["f.item", "f.element", "c.name", "c.ticker"]].duplicated()==False)[0], dtype="int")
scenario1 = scenario_df1[["f.item", "f.element", "c.name", "c.ticker"]].iloc[index_dup]
scenario1.index = np.arange(scenario1.shape[0])

#scenario2のレコードを抽出
index_dup = np.array(np.where(scenario_df2[["f1.item", "f1.element", "f2.item", 
                                            "f2.element", "c.name", "c.ticker"]].duplicated()==False)[0], dtype="int")
scenario2 = scenario_df2[["f1.item", "f1.element", "f2.item", "f2.element", "c.name", "c.ticker"]].iloc[index_dup]
scenario2.index = np.arange(scenario2.shape[0])

#factorのレコードを抽出
index_dup = np.array(np.where(factor_df[["f1.item", "f1.element", "f2.item", "f2.element"]].duplicated()==False)[0], dtype="int")
factor = factor_df[["f1.item", "f1.element", "f2.item", "f2.element"]].iloc[index_dup]
factor.index = np.arange(factor.shape[0])

In [6]:
##要因および企業のidを生成
#itemのidを作成
temp = pd.concat((scenario1["f.item"], scenario2["f1.item"], 
                  scenario2["f2.item"], factor["f1.item"], factor["f2.item"]), axis=0)
temp_freq = temp.value_counts()
index = np.where(temp_freq >= 3)[0]
target = np.array(temp_freq.index[index]); item_freq = temp_freq.iloc[index]
temp = temp.iloc[np.where(np.in1d(np.array(temp), target))[0]]
item_entity = np.append(np.unique(temp)[1:], "")
item_id = np.arange(item_entity.shape[0])
item = item_entity.shape[0]-1
item_df = pd.DataFrame({"id": item_id, "item": item_entity})

#elementのidを作成
temp = pd.concat((scenario1["f.element"], scenario2["f1.element"], 
                  scenario2["f2.element"], factor["f1.element"], factor["f2.element"]), axis=0)
temp_freq = temp.value_counts()
index = np.where(temp_freq >= 3)[0]
target = np.array(temp_freq.index[index]); element_freq = temp_freq.iloc[index]
temp = temp.iloc[np.where(np.in1d(np.array(temp), target))[0]]
element_entity = np.append(np.unique(temp)[1:], "")
element_id = np.arange(element_entity.shape[0])
element = element_entity.shape[0]-1
element_df = pd.DataFrame({"id": element_id, "element": element_entity})

#企業のidを作成
temp = pd.concat((scenario1[["c.name", "c.ticker"]], scenario2[["c.name", "c.ticker"]]), axis=0)
temp_freq = temp["c.ticker"].value_counts()
index = np.where(temp_freq >= 5)[0]
target = np.array(temp_freq.index[index]); company_freq = temp_freq.iloc[index]
temp = temp.iloc[np.where(np.in1d(np.array(temp["c.ticker"]), target))[0]]
temp.index = np.arange(temp.shape[0])
company_df = temp.iloc[np.where(temp.duplicated()==False)[0]]
company_df = company_df.sort_values("c.ticker")
company_df.index = np.arange(company_df.shape[0])
company_df["id"] = np.arange(company_df.shape[0])
company_df = company_df[["id", "c.ticker", "c.name"]]
company_df.columns = ["id", "ticker", "company"]
company_id = np.array(company_df["id"], dtype="int")

#頻度のデータフレームを作成
company_freq = pd.DataFrame({"ticker": np.array(company_freq.index, dtype="int"), "freq": np.array(company_freq, dtype="int")})
company_freq = pd.merge(company_freq, company_df, on="ticker", how="left")[["id", "ticker", "company", "freq"]]
item_freq = pd.DataFrame({"item": np.array(item_freq.index), "freq": np.array(item_freq, dtype="int")})
item_freq = pd.merge(item_freq, item_df, on="item", how="left")[["id", "item", "freq"]]
element_freq = pd.DataFrame({"element": np.array(element_freq.index), "freq": np.array(element_freq, dtype="int")})
element_freq = pd.merge(element_freq, element_df, on="element", how="left")[["id", "element", "freq"]]

In [7]:
##要因-企業間のネットワークデータを生成
#要因-企業間のネットワークを生成
item_flag = pd.merge(scenario1[["f.item"]], item_df[:item_df.shape[0]-1], left_on="f.item", right_on="item", how="left")[["id", "item"]]
element_flag = pd.merge(scenario1[["f.element"]], element_df[:element_df.shape[0]-1], 
                        left_on="f.element", right_on="element", how="left")[["id", "element"]]
company_flag = pd.merge(scenario1[["c.ticker"]], company_df, 
                        left_on="c.ticker", right_on="ticker", how="left")[["id", "ticker", "company"]]
index_get = np.array(np.where((pd.isna(item_flag["id"])==False) & 
                              (pd.isna(element_flag["id"])==False) & (pd.isna(company_flag["id"])==False))[0], dtype="int")
item_fc1 = np.array(item_flag["id"].iloc[index_get], dtype="int16")
element_fc1 = np.array(element_flag["id"].iloc[index_get], dtype="int16")
company_fc1 = np.array(company_flag["id"].iloc[index_get], dtype="int16")
n1 = index_get.shape[0]

#データフレームを作成
temp_item = item_df.iloc[item_fc1]
temp_item.columns = ["item_id", "item"]; temp_item.index = np.arange(temp_item.shape[0])
temp_element = element_df.iloc[element_fc1]
temp_element.columns = ["element_id", "element"]; temp_element.index = np.arange(temp_element.shape[0])
temp_company = company_df.iloc[company_fc1]
temp_company.columns = ["company_id", "ticker", "company"]; temp_company.index = np.arange(temp_company.shape[0])
fc = pd.concat((temp_item, temp_element, temp_company), axis=1)
del temp_item, temp_element, temp_company

In [8]:
#要因-要因間のネットワークを生成
item_flag1 = pd.merge(factor[["f1.item"]], item_df[:item_df.shape[0]-1], left_on="f1.item", right_on="item", how="left")[["id", "item"]]
item_flag2 = pd.merge(factor[["f2.item"]], item_df[:item_df.shape[0]-1], left_on="f2.item", right_on="item", how="left")[["id", "item"]]
element_flag1 = pd.merge(factor[["f1.element"]], element_df[:element_df.shape[0]-1], 
                         left_on="f1.element", right_on="element", how="left")[["id", "element"]] 
element_flag2 = pd.merge(factor[["f2.element"]], element_df[:element_df.shape[0]-1], 
                         left_on="f2.element", right_on="element", how="left")[["id", "element"]]
index_get = np.array(np.where((pd.isna(item_flag1["id"])==False) & (pd.isna(item_flag2["id"])==False) &
                              (pd.isna(element_flag1["id"])==False) & (pd.isna(element_flag2["id"])==False))[0], dtype="int")
item_ff11 = np.array(item_flag1["id"].iloc[index_get], dtype="int16")
item_ff12 = np.array(item_flag2["id"].iloc[index_get], dtype="int16")
element_ff11 = np.array(element_flag1["id"].iloc[index_get], dtype="int16")
element_ff12 = np.array(element_flag2["id"].iloc[index_get], dtype="int16")
n2 = index_get.shape[0]

#データフレームを作成
temp_item1 = item_df.iloc[item_ff11]
temp_item1.columns = ["item_id1", "item1"]; temp_item1.index = np.arange(temp_item1.shape[0])
temp_item2 = item_df.iloc[item_ff12]
temp_item2.columns = ["item_id2", "item2"]; temp_item2.index = np.arange(temp_item2.shape[0])
temp_element1 = element_df.iloc[element_ff11]
temp_element1.columns = ["element_id1", "element1"]; temp_element1.index = np.arange(temp_element1.shape[0])
temp_element2 = element_df.iloc[element_ff12]
temp_element2.columns = ["element_id2", "element2"]; temp_element2.index = np.arange(temp_element2.shape[0])
ff = pd.concat((temp_item1, temp_element1, temp_item2, temp_element2), axis=1)
del temp_item1, temp_item2, temp_element1, temp_element2

In [9]:
#要因-要因-企業間のネットワークを生成
item_flag1 = pd.merge(scenario2[["f1.item"]], item_df[:item_df.shape[0]-1], left_on="f1.item", right_on="item", how="left")[["id", "item"]]
item_flag2 = pd.merge(scenario2[["f2.item"]], item_df[:item_df.shape[0]-1], left_on="f2.item", right_on="item", how="left")[["id", "item"]]
element_flag1 = pd.merge(scenario2[["f1.element"]], element_df[:element_df.shape[0]-1], 
                         left_on="f1.element", right_on="element", how="left")[["id", "element"]]
element_flag2 = pd.merge(scenario2[["f2.element"]], element_df[:element_df.shape[0]-1], 
                         left_on="f2.element", right_on="element", how="left")[["id", "element"]]
company_flag = pd.merge(scenario2[["c.ticker"]], company_df,
                        left_on="c.ticker", right_on="ticker", how="left")[["id", "ticker", "company"]]
index_get = np.array(np.where((pd.isna(item_flag1["id"])==False) & (pd.isna(item_flag2["id"])==False) &
                              (pd.isna(element_flag1["id"])==False) & (pd.isna(element_flag2["id"])==False) & 
                              (pd.isna(company_flag["id"])==False))[0], dtype="int")
item_ffc11 = np.array(item_flag1["id"].iloc[index_get], dtype="int16")
item_ffc12 = np.array(item_flag2["id"].iloc[index_get], dtype="int16")
element_ffc11 = np.array(element_flag1["id"].iloc[index_get], dtype="int16")
element_ffc12 = np.array(element_flag2["id"].iloc[index_get], dtype="int16")
company_ffc1 = np.array(company_flag["id"].iloc[index_get], dtype="int16")
n3 = index_get.shape[0]

#データフレームを作成
temp_item1 = item_df.iloc[item_ffc11]
temp_item1.columns = ["item_id1", "item1"]; temp_item1.index = np.arange(temp_item1.shape[0])
temp_item2 = item_df.iloc[item_ffc12]
temp_item2.columns = ["item_id2", "item2"]; temp_item2.index = np.arange(temp_item2.shape[0])
temp_element1 = element_df.iloc[element_ffc11]
temp_element1.columns = ["element_id1", "element1"]; temp_element1.index = np.arange(temp_element1.shape[0])
temp_element2 = element_df.iloc[element_ffc12]
temp_element2.columns = ["element_id2", "element2"]; temp_element2.index = np.arange(temp_element2.shape[0])
temp_company = company_df.iloc[company_ffc1]
temp_company.columns = ["company_id", "ticker", "company"]; temp_company.index = np.arange(temp_company.shape[0])
ffc = pd.concat((temp_item1, temp_element1, temp_item2, temp_element2, temp_company), axis=1)
del temp_item1, temp_item2, temp_element1, temp_element2, temp_company

In [10]:
##要因-企業-要因のネットワークデータを生成
#要因と企業を抽出
item_flag = pd.merge(scenario1[["f.item"]], item_df, left_on="f.item", right_on="item", how="left")[["id", "item"]]
element_flag = pd.merge(scenario1[["f.element"]], element_df, left_on="f.element", right_on="element", how="left")[["id", "element"]]
company_flag = pd.merge(scenario1[["c.ticker"]], company_df, 
                        left_on="c.ticker", right_on="ticker", how="left")[["id", "ticker", "company"]]
index_get = np.array(np.where((pd.isna(item_flag["id"])==False) & 
                              (pd.isna(element_flag["id"])==False) & (pd.isna(company_flag["id"])==False))[0], dtype="int")

#データの格納用リスト
get_company = np.unique(company_fc1)
n = get_company.shape[0]
company_list = [i for i in range(n)]
item_list1 = [i for i in range(n)]
item_list2 = [i for i in range(n)]
element_list1 = [i for i in range(n)]
element_list2 = [i for i in range(n)]

for i in range(n):
    #対象のitemとelementを抽出
    index = np.array(np.where(company_fc1==get_company[i])[0], dtype="int")
    get_item = item_fc1[index]
    get_element = element_fc1[index]
    if get_element.shape[0] <= 1:
        company_list[i] = np.array([], dtype="int16")
        item_list1[i] = np.array([], dtype="int16")
        item_list2[i] = np.array([], dtype="int16")
        element_list1[i] = np.array([], dtype="int16")
        element_list2[i] = np.array([], dtype="int16")
        continue

    #組み合わせを作成
    m = get_item.shape[0]
    get_combine1 = np.repeat(np.arange(m-1), np.arange(m-1)[::-1]+1)
    index_del = np.array([], dtype="int")
    get_combine2 = np.array([], dtype="int")
    for j in range(m-1):
        index_del = np.append(index_del, j) 
        get_combine2 = np.append(get_combine2, np.delete(np.arange(m), index_del))

    #リストに格納
    company_list[i] = np.repeat(get_company[i], get_combine1.shape[0])
    item_list1[i] = get_item[get_combine1]
    item_list2[i] = get_item[get_combine2]
    element_list1[i] = get_element[get_combine1]
    element_list2[i] = get_element[get_combine2]
    
#リストを変換
company_fcf1 = np.array(list(itertools.chain(*[company_list[i] for i in range(n)])))
item_fcf11 = np.array(list(itertools.chain(*[item_list1[i] for i in range(n)])))
item_fcf12 = np.array(list(itertools.chain(*[item_list2[i] for i in range(n)])))
element_fcf11 = np.array(list(itertools.chain(*[element_list1[i] for i in range(n)])))
element_fcf12 = np.array(list(itertools.chain(*[element_list2[i] for i in range(n)])))
n4 = company_fcf1.shape[0]

#データフレームを作成
temp_item1 = item_df.iloc[item_fcf11]
temp_item1.columns = ["item_id1", "item1"]; temp_item1.index = np.arange(temp_item1.shape[0])
temp_item2 = item_df.iloc[item_fcf12]
temp_item2.columns = ["item_id2", "item2"]; temp_item2.index = np.arange(temp_item2.shape[0])
temp_element1 = element_df.iloc[element_fcf11]
temp_element1.columns = ["element_id1", "element1"]; temp_element1.index = np.arange(temp_element1.shape[0])
temp_element2 = element_df.iloc[element_fcf12]
temp_element2.columns = ["element_id2", "element2"]; temp_element2.index = np.arange(temp_element2.shape[0])
temp_company = company_df.iloc[company_fcf1]
temp_company.columns = ["company_id", "ticker", "company"]; temp_company.index = np.arange(temp_company.shape[0])
fcf = pd.concat((temp_company, temp_item1, temp_element1, temp_item2, temp_element2), axis=1)
del item_list1, item_list2, element_list1, element_list2, company_list
del temp_item1, temp_item2, temp_element1, temp_element2, temp_company

In [11]:
##企業-要因-企業のネットワークデータを生成
#要因と企業を抽出
item_flag = pd.merge(scenario1[["f.item"]], item_df, left_on="f.item", right_on="item", how="left")[["id", "item"]]
element_flag = pd.merge(scenario1[["f.element"]], element_df, left_on="f.element", right_on="element", how="left")[["id", "element"]]
company_flag = pd.merge(scenario1[["c.ticker"]], company_df, 
                        left_on="c.ticker", right_on="ticker", how="left")[["id", "ticker", "company"]]
index_get = np.array(np.where((pd.isna(item_flag["id"])==False) & 
                              (pd.isna(element_flag["id"])==False) & (pd.isna(company_flag["id"])==False))[0], dtype="int")

#データの格納用リスト
get_item = np.unique(item_fc1); get_item = get_item[:get_item.shape[0]-1]
n = get_item.shape[0]
item_list = [i for i in range(n)]
company_list1 = [i for i in range(n)]
company_list2 = [i for i in range(n)]

for i in range(n):
    #対象の企業を抽出
    index = np.array(np.where(item_fc1==get_item[i])[0], dtype="int")
    get_company = company_fc1[index]
    if get_element.shape[0] <= 1:
        item_list[i] = np.array([], dtype="int16")
        company_list1[i] = np.array([], dtype="int16")
        company_list2[i] = np.array([], dtype="int16")
        continue

    #組み合わせを作成
    m = get_company.shape[0]
    get_combine1 = np.repeat(np.arange(m-1), np.arange(m-1)[::-1]+1)
    index_del = np.array([], dtype="int")
    get_combine2 = np.array([], dtype="int")
    for j in range(m-1):
        index_del = np.append(index_del, j) 
        get_combine2 = np.append(get_combine2, np.delete(np.arange(m), index_del))

    #リストに格納
    item_list[i] = np.repeat(get_item[i], get_combine1.shape[0])
    company_list1[i] = get_company[get_combine1]
    company_list2[i] = get_company[get_combine2]
    
#リストを変換
item_cfc1 = np.array(list(itertools.chain(*[item_list[i] for i in range(n)])))
company_cfc11 = np.array(list(itertools.chain(*[company_list1[i] for i in range(n)])))
company_cfc12 = np.array(list(itertools.chain(*[company_list2[i] for i in range(n)])))
n5 = item_cfc1.shape[0]

#データフレームを作成
temp_item = item_df.iloc[item_cfc1]
temp_item.columns = ["item_id", "item"]; temp_item.index = np.arange(temp_item.shape[0])
temp_company1 = company_df.iloc[company_cfc11]
temp_company1.columns = ["company_id1", "ticker1", "company1"]; temp_company1.index = np.arange(temp_company1.shape[0])
temp_company2 = company_df.iloc[company_cfc12]
temp_company2.columns = ["company_id2", "ticker2", "company2"]; temp_company2.index = np.arange(temp_company2.shape[0])
cfc = pd.concat((temp_company1, temp_company2, temp_item), axis=1)
del item_list, company_list1, company_list2
del temp_item, temp_company1, temp_company2

In [11]:
#データフレームを出力
fc.to_excel("D:/Statistics/data/scenario_link/factor-company1.xlsx")
ff.to_excel("D:/Statistics/data/scenario_link/factor-factor1.xlsx")
ffc.to_excel("D:/Statistics/data/scenario_link/factor-factor-company1.xlsx")
fcf.to_excel("D:/Statistics/data/scenario_link/factor-company-factor1.xlsx")
cfc.to_excel("D:/Statistics/data/scenario_link/company-factor-company1.xlsx")

In [12]:
##負例サンプリングにより新しいサンプルを生成
#サンプリング確率を設定
s = 2
item_prob = np.power(item_freq["freq"], 0.75) / np.sum(np.power(item_freq["freq"], 0.75))
element_prob = np.power(element_freq["freq"], 0.75) / np.sum(np.power(element_freq["freq"], 0.75))
company_prob = np.power(company_freq["freq"], 0.75) / np.sum(np.power(company_freq["freq"], 0.75))


#要因-企業間の負例をサンプリング
item_fc0 = np.array(np.dot(np.random.multinomial(1, item_prob, n1*s), np.arange(item_prob.shape[0])), dtype="int16")
item_fc0 = np.array(item_freq["id"].iloc[item_fc0], dtype="int16")
element_fc0 = np.array(np.dot(np.random.multinomial(1, element_prob, n1*s), np.arange(element_prob.shape[0])), dtype="int16")
element_fc0 = np.array(element_freq["id"].iloc[element_fc0], dtype="int16")
company_fc0 = np.array(np.dot(np.random.multinomial(1, company_prob, n1*s), np.arange(company_prob.shape[0])), dtype="int16")
company_fc0 = np.array(company_freq["id"].iloc[company_fc0], dtype="int16")

#要因-要因間の負例をサンプリング
item_ff01 = np.array(np.dot(np.random.multinomial(1, item_prob, n2*s), np.arange(item_prob.shape[0])), dtype="int16")
item_ff01 = np.array(item_freq["id"].iloc[item_ff01], dtype="int16")
item_ff02 = np.array(np.dot(np.random.multinomial(1, item_prob, n2*s), np.arange(item_prob.shape[0])), dtype="int16")
item_ff02 = np.array(item_freq["id"].iloc[item_ff02], dtype="int16")
element_ff01 = np.array(np.dot(np.random.multinomial(1, element_prob, n2*s), np.arange(element_prob.shape[0])), dtype="int16")
element_ff01 = np.array(element_freq["id"].iloc[element_ff01], dtype="int16")
element_ff02 = np.array(np.dot(np.random.multinomial(1, element_prob, n2*s), np.arange(element_prob.shape[0])), dtype="int16")
element_ff02 = np.array(element_freq["id"].iloc[element_ff02], dtype="int16")

#要因-要因-企業間の負例をサンプリング
item_ffc01 = np.array(np.dot(np.random.multinomial(1, item_prob, n3*s), np.arange(item_prob.shape[0])), dtype="int16")
item_ffc01 = np.array(item_freq["id"].iloc[item_ffc01], dtype="int16")
item_ffc02 = np.array(np.dot(np.random.multinomial(1, item_prob, n3*s), np.arange(item_prob.shape[0])), dtype="int16")
item_ffc02 = np.array(item_freq["id"].iloc[item_ffc02], dtype="int16")
element_ffc01 = np.array(np.dot(np.random.multinomial(1, element_prob, n3*s), np.arange(element_prob.shape[0])), dtype="int16")
element_ffc01 = np.array(element_freq["id"].iloc[element_ffc01], dtype="int16")
element_ffc02 = np.array(np.dot(np.random.multinomial(1, element_prob, n3*s), np.arange(element_prob.shape[0])), dtype="int16")
element_ffc02 = np.array(element_freq["id"].iloc[element_ffc02], dtype="int16")
company_ffc0 = np.array(np.dot(np.random.multinomial(1, company_prob, n3*s), np.arange(company_prob.shape[0])), dtype="int16")
company_ffc0 = np.array(company_freq["id"].iloc[company_ffc0], dtype="int16")

#要因-企業-要因間の負例をサンプリング
item_fcf01 = np.array(np.dot(np.random.multinomial(1, item_prob, n4*s), np.arange(item_prob.shape[0])), dtype="int16")
item_fcf01 = np.array(item_freq["id"].iloc[item_fcf01], dtype="int16")
item_fcf02 = np.array(np.dot(np.random.multinomial(1, item_prob, n4*s), np.arange(item_prob.shape[0])), dtype="int16")
item_fcf02 = np.array(item_freq["id"].iloc[item_fcf02], dtype="int16")
element_fcf01 = np.array(np.dot(np.random.multinomial(1, element_prob, n4*s), np.arange(element_prob.shape[0])), dtype="int16")
element_fcf01 = np.array(element_freq["id"].iloc[element_fcf01], dtype="int16")
element_fcf02 = np.array(np.dot(np.random.multinomial(1, element_prob, n4*s), np.arange(element_prob.shape[0])), dtype="int16")
element_fcf02 = np.array(element_freq["id"].iloc[element_fcf02], dtype="int16")
company_fcf0 = np.array(np.dot(np.random.multinomial(1, company_prob, n4*s), np.arange(company_prob.shape[0])), dtype="int16")
company_fcf0 = np.array(company_freq["id"].iloc[company_fcf0], dtype="int16")

#企業-要因-企業間の負例をサンプリング
item_cfc0 = np.array(np.dot(np.random.multinomial(1, item_prob, n5*s), np.arange(item_prob.shape[0])), dtype="int16")
item_cfc0 = np.array(item_freq["id"].iloc[item_cfc0], dtype="int16")
company_cfc01 = np.array(np.dot(np.random.multinomial(1, company_prob, n5*s), np.arange(company_prob.shape[0])), dtype="int16")
company_cfc01 = np.array(company_freq["id"].iloc[company_cfc01], dtype="int16")
company_cfc02 = np.array(np.dot(np.random.multinomial(1, company_prob, n5*s), np.arange(company_prob.shape[0])), dtype="int16")
company_cfc02 = np.array(company_freq["id"].iloc[company_cfc02], dtype="int16")

In [13]:
##応答変数を設定
#要因-企業間の応答変数を定義
item_fc = np.append(item_fc1, item_fc0)
element_fc = np.append(element_fc1, element_fc0)
company_fc = np.append(company_fc1, company_fc0)
y1 = np.append(np.repeat(1, n1), np.repeat(0, n1*s))
N1 = y1.shape[0]

#要因-要因間の応答変数を定義
item_ff1 = np.append(item_ff11, item_ff01)
item_ff2 = np.append(item_ff12, item_ff02)
element_ff1 = np.append(element_ff11, element_ff01)
element_ff2 = np.append(element_ff12, element_ff02)
y2 = np.append(np.repeat(1, n2), np.repeat(0, n2*s))
N2 = y2.shape[0]

#要因ー要因-企業間の応答変数を定義
item_ffc1 = np.append(item_ffc11, item_ffc01)
item_ffc2 = np.append(item_ffc12, item_ffc02)
element_ffc1 = np.append(element_ffc11, element_ffc01)
element_ffc2 = np.append(element_ffc12, element_ffc02)
company_ffc = np.append(company_ffc1, company_ffc0)
y3 = np.append(np.repeat(1, n3), np.repeat(0, n3*s))
N3 = y3.shape[0]

#要因ー企業-要因間の応答変数を定義
item_fcf1 = np.append(item_fcf11, item_fcf01)
item_fcf2 = np.append(item_fcf12, item_fcf02)
element_fcf1 = np.append(element_fcf11, element_fcf01)
element_fcf2 = np.append(element_fcf12, element_fcf02)
company_fcf = np.append(company_fcf1, company_fcf0)
y4 = np.append(np.repeat(1, n4), np.repeat(0, n4*s))
N4 = y4.shape[0]

#企業-要因-企業間の応答変数を定義
item_cfc = np.append(item_cfc1, item_cfc0)
company_cfc1 = np.append(company_cfc11, company_cfc01)
company_cfc2 = np.append(company_cfc12, company_cfc02)
y5 = np.append(np.repeat(1, n5), np.repeat(0, n5*s))
N5 = y5.shape[0]

In [14]:
##学習データとテストデータに分割
#要因-企業間のデータを分割
split = 4/5
index_learn = np.random.choice(N1, int(split * N1), replace=False)
index_test = np.delete(np.arange(N1), index_learn)
item_fc1 = item_fc[index_learn]; item_fc2 = item_fc[index_test]
element_fc1 = element_fc[index_learn]; element_fc2 = element_fc[index_test]
company_fc1 = company_fc[index_learn]; company_fc2 = company_fc[index_test]
y11 = y1[index_learn]; y12 = y1[index_test]
N11 = index_learn.shape[0]; N12 = index_test.shape[0]

#要因-要因間のデータを分割
index_learn = np.random.choice(N2, int(split * N2), replace=False)
index_test = np.delete(np.arange(N2), index_learn)
item_ff11 = item_ff1[index_learn]; item_ff12 = item_ff1[index_test]
item_ff21 = item_ff2[index_learn]; item_ff22 = item_ff2[index_test]
element_ff11 = element_ff1[index_learn]; element_ff12 = element_ff1[index_test]
element_ff21 = element_ff2[index_learn]; element_ff22 = element_ff2[index_test]
y21 = y2[index_learn]; y22 = y2[index_test]
N21 = index_learn.shape[0]; N22 = index_test.shape[0]

#要因-要因-企業間のデータを分割
index_learn = np.random.choice(N3, int(split * N3), replace=False)
index_test = np.delete(np.arange(N3), index_learn)
item_ffc11 = item_ffc1[index_learn]; item_ff12 = item_ffc1[index_test]
item_ffc21 = item_ffc2[index_learn]; item_ff22 = item_ffc2[index_test]
element_ffc11 = element_ffc1[index_learn]; element_ffc12 = element_ffc1[index_test]
element_ffc21 = element_ffc2[index_learn]; element_ffc22 = element_ffc2[index_test]
company_ffc1 = company_ffc[index_learn]; company_ffc2 = company_ffc[index_test]
y31 = y3[index_learn]; y32 = y3[index_test]
N31 = index_learn.shape[0]; N32 = index_test.shape[0]

#要因-企業-要因間のデータを分割
index_learn = np.random.choice(N4, int(split * N4), replace=False)
index_test = np.delete(np.arange(N4), index_learn)
item_fcf11 = item_fcf1[index_learn]; item_fcf12 = item_fcf1[index_test]
item_fcf21 = item_fcf2[index_learn]; item_fcf22 = item_fcf2[index_test]
element_fcf11 = element_fcf1[index_learn]; element_fcf12 = element_fcf1[index_test]
element_fcf21 = element_fcf2[index_learn]; element_fcf22 = element_fcf2[index_test]
company_fcf1 = company_fcf[index_learn]; company_fcf2 = company_fcf[index_test]
y41 = y4[index_learn]; y42 = y4[index_test]
N41 = index_learn.shape[0]; N42 = index_test.shape[0]

#企業-要因-企業間のデータを分割
index_learn = np.random.choice(N5, int(split * N5), replace=False)
index_test = np.delete(np.arange(N5), index_learn)
item_cfc1 = item_cfc[index_learn]; item_cfc2 = item_cfc[index_test]
company_cfc11 = company_cfc1[index_learn]; company_cfc12 = company_cfc1[index_test]
company_cfc21 = company_cfc2[index_learn]; company_cfc22 = company_cfc2[index_test]
y51 = y5[index_learn]; y52 = y5[index_test]
N51 = index_learn.shape[0]; N52 = index_test.shape[0]
N = [N11, N21, N31, N41, N51]

In [15]:
##インデックスを設定
#要因と企業のユニーク数
item_n = item_df.shape[0]-1
element_n = element_df.shape[0]-1
company_n = company_df.shape[0]

#itemのインデックスを設定
item_list_fc = [i for i in range(item_n)]
item_list_ff1 = [i for i in range(item_n)]
item_list_ff2 = [i for i in range(item_n)]
item_list_ffc1 = [i for i in range(item_n)]
item_list_ffc2 = [i for i in range(item_n)]
item_list_fcf1 = [i for i in range(item_n)]
item_list_fcf2 = [i for i in range(item_n)]
item_list_cfc = [i for i in range(item_n)]
for i in range(item_n):
    item_list_fc[i] = np.array(np.where(item_fc1==i)[0], dtype="int")
    item_list_ff1[i] = np.array(np.where(item_ff11==i)[0], dtype="int")
    item_list_ff2[i] = np.array(np.where(item_ff21==i)[0], dtype="int")
    item_list_ffc1[i] = np.array(np.where(item_ffc11==i)[0], dtype="int")
    item_list_ffc2[i] = np.array(np.where(item_ffc21==i)[0], dtype="int")
    item_list_fcf1[i] = np.array(np.where(item_fcf11==i)[0], dtype="int")
    item_list_fcf2[i] = np.array(np.where(item_fcf21==i)[0], dtype="int")
    item_list_cfc[i] = np.array(np.where(item_cfc1==i)[0], dtype="int")
    
#elementのインデックスを設定
element_list_fc = [i for i in range(element_n)]
element_list_ff1 = [i for i in range(element_n)]
element_list_ff2 = [i for i in range(element_n)]
element_list_ffc1 = [i for i in range(element_n)]
element_list_ffc2 = [i for i in range(element_n)]
element_list_fcf1 = [i for i in range(element_n)]
element_list_fcf2 = [i for i in range(element_n)]
for i in range(element_n):
    element_list_fc[i] = np.array(np.where(element_fc1==i)[0], dtype="int")
    element_list_ff1[i] = np.array(np.where(element_ff11==i)[0], dtype="int")
    element_list_ff2[i] = np.array(np.where(element_ff21==i)[0], dtype="int")
    element_list_ffc1[i] = np.array(np.where(element_ffc11==i)[0], dtype="int")
    element_list_ffc2[i] = np.array(np.where(element_ffc21==i)[0], dtype="int")
    element_list_fcf1[i] = np.array(np.where(element_fcf11==i)[0], dtype="int")
    element_list_fcf2[i] = np.array(np.where(element_fcf21==i)[0], dtype="int")
    
#企業のインデックスを設定
company_list_fc = [i for i in range(company_n)]
company_list_ffc = [i for i in range(company_n)]
company_list_fcf = [i for i in range(company_n)]
company_list_cfc1 = [i for i in range(company_n)]
company_list_cfc2 = [i for i in range(company_n)]
for i in range(company_n):
    company_list_fc[i] = np.array(np.where(company_fc1==i)[0], dtype="int")
    company_list_ffc[i] = np.array(np.where(company_ffc1==i)[0], dtype="int")
    company_list_fcf[i] = np.array(np.where(company_fcf1==i)[0], dtype="int")
    company_list_cfc1[i] = np.array(np.where(company_cfc11==i)[0], dtype="int")
    company_list_cfc2[i] = np.array(np.where(company_cfc21==i)[0], dtype="int")

In [16]:
####マルコフ連鎖モンテカルロ法でパラメータを推定####
##切断正規分布の乱数を発生させる関数
def rtnorm(mu, sigma, a, b, n):
    FA = norm.cdf(a, mu, sigma)
    FB = norm.cdf(b, mu, sigma)
    return norm.ppf(np.random.uniform(0, 1, n)*(FB-FA)+FA, mu, sigma)

In [17]:
##ベイジアン多変量回帰モデルをギブスサンプリングする関数
def rmultireg(Y, X, inv_XXV, XY, Cov, ADelta, Deltabar, V, nu, n, col, k):
    #事後分布のパラメータを設定
    beta_mu = np.dot(inv_XXV, XY + np.dot(ADelta, Deltabar)).T.reshape(-1)   #平均ベクトル
    sigma = np.kron(Cov, inv_XXV)   #分散共分散行列

    #パラメータをサンプリング
    beta_vec = np.random.multivariate_normal(beta_mu, sigma, 1)
    beta = beta_vec.reshape(col, k, order='F')   #回帰行列に変換

    
    ##逆ウィシャート分布から分散共分散行列をサンプリング
    #モデル誤差を設定
    mu = np.dot(X, beta)
    er = Y - mu

    #逆ウィシャート分布のパラメータ
    IW_R = np.dot(er.T, er) + V
    Sn = n + nu

    #パラメータをサンプリング
    Cov = scipy.stats.invwishart.rvs(Sn, IW_R, 1)
    return beta, Cov

In [18]:
##アルゴリズムの設定
type_n = 5
k = 10
k_vec = np.repeat(1, k)
R = 1000
keep = 2
burnin = int(100/keep)
iter = 0
disp = 10

In [19]:
##事前分布の設定
#階層モデルの事前分布
delta = np.repeat(0, k)
V = 0.1 * np.diag(np.ones(k))
nu = 1
s01 = 0.1
s01 = 0.1

#モデルパラメータの事前分布
alpha = np.repeat(0.0, k)
tau = 100 
s02 = 1.0
v02 = 1.0

In [20]:
##初期値の設定
#階層モデルの初期値
Cov_v = np.array(np.diag(np.repeat(0.5, k)), dtype="float32"); inv_Cov_v = np.linalg.inv(Cov_v)
Cov_i = np.array(np.diag(np.repeat(0.5, k)), dtype="float32"); inv_Cov_i = np.linalg.inv(Cov_i)
Cov_e = np.array(np.diag(np.repeat(0.5, k)), dtype="float32"); inv_Cov_e = np.linalg.inv(Cov_e)

#モデルパラメータの初期値
Sigma = np.array([1.0], dtype="float32")
beta = np.array(np.repeat(0.0, type_n), dtype="float32")
theta_v = np.random.multivariate_normal(alpha, np.diag(np.repeat(0.1, k)), company_n)
theta_i = np.array(np.vstack((np.random.multivariate_normal(alpha, np.diag(np.repeat(0.1, k)), item_n), np.repeat(1, k))), dtype="float32")
theta_e = np.array(np.vstack((np.random.multivariate_normal(alpha, np.diag(np.repeat(0.1, k)), element_n), np.repeat(1, k))), dtype="float32")
omega_fc = np.array(np.random.normal(0, 0.1, k*k).reshape(k, k), dtype="float32")
omega_ff = np.array(np.random.normal(0, 0.1, k*k).reshape(k, k), dtype="float32")
omega_ffc = np.array(np.random.normal(0, 0.1, k*k*k).reshape(k, k, k), dtype="float32")
omega_fcf = np.array(np.random.normal(0, 0.1, k*k*k).reshape(k, k, k), dtype="float32")
omega_cfc = np.array(np.random.normal(0, 0.1, k*k*k).reshape(k, k, k), dtype="float32")

In [21]:
##データの設定
#切断領域を定義
rho = 10.0
y = [y11, y21, y31, y41, y51]
a = [j for j in range(type_n)]
b = [j for j in range(type_n)]
for j in range(type_n):
    a[j] = np.array((1-y[j] )*(-rho) + y[j]*0, dtype="float32")
    b[j] = np.array(y[j]*rho + (1-y[j])*0, dtype="float32")
    
#特徴行列の割当インデックス
allocation_theta = np.repeat(np.arange(k), k)
allocation_omega = np.array([np.delete(np.arange(k), j).tolist() for j in range(k)])

In [22]:
####ギブスサンプリングでパラメータをサンプリング####

##切断正規分布から潜在効用を生成
#特徴ベクトルをidで対応付ける
theta_item1 = theta_i[item_fc1, ]; theta_element1 = theta_e[element_fc1, ]; theta_company1 = theta_v[company_fc1, ]
theta_item21 = theta_i[item_ff11, ]; theta_item22 = theta_i[item_ff21, ]
theta_element21 = theta_e[element_ff11, ]; theta_element22 = theta_v[element_ff21, ]
theta_item31 = theta_i[item_ffc11, ]; theta_item32 = theta_i[item_ffc21, ]
theta_element31 = theta_e[element_ffc11, ]; theta_element32 = theta_e[element_ffc21, ]; theta_company3 = theta_v[company_ffc1, ]
theta_item41 = theta_i[item_fcf11, ]; theta_item42 = theta_i[item_fcf21, ]
theta_element41 = theta_e[element_fcf11, ]; theta_element42 = theta_e[element_fcf21, ]; theta_company4 = theta_v[company_fcf1, ]
theta_item5 = theta_i[item_cfc1, ]; theta_company51 = theta_v[company_cfc11, ]; theta_company52 = theta_v[company_cfc21, ]

#タッカー分解の期待値
uv1 = np.dot(np.dot(theta_item1 * theta_element1, omega_fc) * theta_company1, k_vec)
uv2 = np.dot(np.dot(theta_item21 * theta_element21, omega_ff) * (theta_item22 * theta_element22), k_vec)
uv3 = np.array(np.repeat(0.0, N31), dtype="float32")
uv4 = np.array(np.repeat(0.0, N41), dtype="float32")
uv5 = np.array(np.repeat(0.0, N51), dtype="float32")
theta_joint31 = theta_item31 * theta_element31; theta_joint32 = theta_item32 * theta_element32
theta_joint41 = theta_item41 * theta_element41; theta_joint42 = theta_item42 * theta_element42
for j in range(k):
    uv3 += np.dot(np.dot(theta_joint31, omega_ffc[:, :, j]) * theta_joint32 * theta_company3[:, j][:, np.newaxis], k_vec)
    uv4 += np.dot(np.dot(theta_joint41, omega_fcf[:, :, j]) * theta_company4 * theta_joint41[:, j][:, np.newaxis], k_vec)
    uv5 += np.dot(np.dot(theta_company51, omega_cfc[:, :, j]) * theta_item5 * theta_company52[:, j][:, np.newaxis], k_vec)
uv = [uv1, uv2, uv3, uv4, uv5]

#潜在効用を生成
mu = [j for j in range(type_n)]
U = [j for j in range(type_n)]
for j in range(type_n):
    mu[j] = beta[j] + uv[j]
    U[j] = np.array(rtnorm(mu[j], Sigma, a[j], b[j], N[j]), dtype="float32")

In [23]:
##モデルの期待値をサンプリング
for j in range(type_n):
    #モデル誤差を定義
    er_y = U[j] - uv[j]

    #正規分布から期待値をサンプリング
    weights = np.power(tau, 2) / (Sigma/N[j] + np.power(tau, 2))
    mu_par = weights * np.mean(er_y)
    beta[j] = np.random.normal(mu_par, weights*Sigma/N[j], 1)

In [29]:
##アイテムの特徴ベクトルをサンプリング


d = np.repeat(1, k)
i = 0
np.sum((np.dot(d, omega_fc) * d) * theta_item1[i, ] * theta_element1[i, ] * theta_company1[i, ])

-0.012834349941271254

In [45]:
i = 3
temp_index = np.sort(np.append(item_list_ff1[i], item_list_ff2[i]))
index_dup = np.array(np.where(pd.Series(temp_index).duplicated())[0])

In [61]:
i = 8
temp_index = np.sort(np.append(element_list_ff1[i], element_list_ff2[i]))
index_dup = np.array(np.where(pd.Series(temp_index).duplicated())[0])

In [62]:
index_dup

array([], dtype=int64)

NameError: name 'element_df' is not defined